In [ ]:
%matplotlib inline
import random
import hashlib
import base64
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt

from dqnroute.utils import *

from sklearn.model_selection import train_test_split
from cycler import cycler

In [ ]:
target_cols = get_target_cols(10)
neighbors_cols = get_neighbors_cols(10)
addr_cols = get_addr_cols(10)
dst_cols = get_dst_cols(10)
amatrix_cols = get_amatrix_cols(10)
left_cols = ['time', 'pkg_id']+neighbors_cols+amatrix_cols+target_cols
new_cols = ['dst', 'addr'] + left_cols
new_cols_2 = ['time', 'pkg_id', 'dst', 'addr', 'neighbour'] + amatrix_cols + ['predict']

In [ ]:
#data = pd.read_csv('../logs/data_generated2_new.csv', names=new_cols)
data = pd.read_csv('../logs/data_generated2_new_one_inp.csv', index_col=0)
data = data.reindex(np.arange(len(data)))

In [ ]:
import gc
gc.collect()

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
%load_ext autoreload
%autoreload 2

from dqnroute.networks import *
from dqnroute.utils import stack_batch
from functools import partial
from tqdm import tqdm_notebook

In [ ]:
def shuffle(df):
    return df.reindex(np.random.permutation(df.index))

def find_first_sublist(seq, sublist, start=0):
    length = len(sublist)
    for index in range(start, len(seq)):
        if seq[index:index+length] == sublist:
            return index, index+length

def replace_sublist(seq, sublist, replacement):
    length = len(replacement)
    index = 0
    for start, end in iter(lambda: find_first_sublist(seq, sublist, index), None):
        seq[start:end] = replacement
        index = start + length
    return seq

def transform_to_one_out(df):
    old_cols = list(df.columns)
    neighbors_cols = [col for col in old_cols if col.startswith('neighbors')]
    target_cols = [col for col in old_cols if col.startswith('predict')]
    
    new_cols = replace_sublist(replace_sublist(old_cols, neighbors_cols, ['neighbour']),
                               target_cols, ['predict'])
    row_ix = 0
    nums = pd.Series(range(len(neighbors_cols)), index=neighbors_cols)
    new_rows_num = df[neighbors_cols].sum().sum()
    df_new = pd.DataFrame(columns=new_cols, index=np.arange(new_rows_num), dtype=np.float32)
    
    for idx, row in tqdm_notebook(df.iterrows(), total=len(df)):
        nbrs = nums[row[neighbors_cols] != 0]
        preds = list(row[target_cols][row != -1000000])        
        new_row_tpl = row.drop(neighbors_cols + target_cols)
        
        for (nbr, pred) in zip(nbrs, preds):
            new_row = new_row_tpl
            new_row['neighbour'] = nbr
            new_row['predict'] = pred
            df_new.loc[row_ix] = new_row
            row_ix += 1
        
    return df_new.reindex(np.arange(len(df_new)))

In [ ]:
def hash_graph(graph):
    if type(graph) != np.ndarray:
        graph = nx.to_numpy_matrix(graph, nodelist=sorted(graph.nodes))
    m = hashlib.sha256()
    m.update(graph.tobytes())
    return base64.b64encode(m.digest()).decode('utf-8')

class CachedEmbedding(Embedding):
    def __init__(self, InnerEmbedding, dim, **kwargs):
        self.dim = dim
        self.InnerEmbedding = InnerEmbedding
        self.inner_kwargs = kwargs
        self.fit_embeddings = {}
        
    def fit(self, graph, **kwargs):
        h = hash_graph(graph)
        if h not in self.fit_embeddings:
            embed = self.InnerEmbedding(dim=self.dim, **self.inner_kwargs)
            embed.fit(graph, **kwargs)
            self.fit_embeddings[h] = embed
    
    def transform(self, graph, idx):
        h = hash_graph(graph)
        return self.fit_embeddings[h].transform(idx)

In [ ]:
def one_or_emb(vals, embedding=None):
    if embedding is None:
        return vals
    return embedding.get_embedding(vals.astype(int))

def qnetwork_batches(net, data, batch_size=64, embedding=None):
    n = net.graph_size
    data_cols = []
    amatrix_cols = get_amatrix_cols(n)
    
    for (tag, dim) in net.add_inputs:
        if tag == 'amatrix':
            data_cols.append(amatrix_cols)
        else:
            data_cols.append(mk_num_list(tag + '_', n))

    for (a, b) in make_batches(data.shape[0], batch_size):
        batch = data[a:b]
        addr = batch['addr'].values
        dst = batch['dst'].values
        nbr = batch['neighbour'].values
        
        if embedding is not None:
            amatrices = batch[amatrix_cols].values
            new_btch = []
            for (addr_, dst_, nbr_, A) in zip(addr, dst, nbr, amatrices):
                A = 10 * A.reshape(n, n)
                embedding.fit(A)
                new_addr = embedding.transform(A, int(addr_))
                new_dst = embedding.transform(A, int(dst_))
                new_nbr = embedding.transform(A, int(nbr_))
                new_btch.append((new_addr, new_dst, new_nbr))
                
            [addr, dst, nbr] = stack_batch(new_btch)
            
        addr_inp = torch.tensor(addr, dtype=torch.float)
        dst_inp = torch.tensor(dst, dtype=torch.float)
        nbr_inp = torch.tensor(nbr, dtype=torch.float)
                
        inputs = tuple(torch.tensor(batch[cols].values, dtype=torch.float)
                       for cols in data_cols)
        output = torch.tensor(batch['predict'].values, dtype=torch.float)
        
        yield ((addr_inp, dst_inp, nbr_inp) + inputs, output)

def qnetwork_pretrain_epoch(net, optimizer, data, **kwargs):
    loss_func = nn.MSELoss()
    for (batch, target) in qnetwork_batches(net, data, **kwargs):
        optimizer.zero_grad()
        output = net(*batch)
        loss = loss_func(output, target.unsqueeze(1))
        loss.backward()
        optimizer.step()
        yield float(loss)
        
def qnetwork_pretrain(net, data, optimizer='rmsprop', epochs=1,
                      save_net=True, **kwargs):
    optimizer = get_optimizer(optimizer)(net.parameters())
    epochs_losses = []
    
    for i in tqdm_notebook(range(epochs)):
        sum_loss = 0
        loss_cnt = 0
        for loss in tqdm_notebook(qnetwork_pretrain_epoch(net, optimizer, data, **kwargs),
                                  desc='epoch {}'.format(i)):
            sum_loss += loss
            loss_cnt += 1
        epochs_losses.append(sum_loss / loss_cnt)
        
    if save_net:
        net.save()
    
    return epochs_losses

In [ ]:
def plot_losses(losses_dict, from_epoch=0, num_epochs=None,
                fsize=16, figsize=(13, 7), title=None):
    if num_epochs is None:
        num_epochs = len(next(iter(losses_dict.values())))
        
    x = range(from_epoch+1, num_epochs+1)
    plt.figure(figsize=figsize)
    for (label, losses) in losses_dict.items():
        plt.plot(x, losses[from_epoch:num_epochs], label=label)
    plt.legend(prop={'size': fsize})
    plt.xlabel('Epoch', fontsize=fsize)
    plt.xticks(x)
    plt.grid()
    plt.ylabel('MSE', fontsize=fsize)
    if title is not None:
        plt.title(title, fontsize=fsize)
    plt.show()

# Feed-forward сети

In [ ]:
seed = 42
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

QNetworkAmatrix = partial(QNetwork, additional_inputs=[{'tag': 'amatrix'}])

In [ ]:
ff_network_amatrix = QNetworkAmatrix(10, activation='tanh', layers=[64, 64])
ff_network_amatrix_64_3 = QNetworkAmatrix(10, layers=[64, 64, 64], activation='tanh')
ff_network_amatrix_128_2 = QNetworkAmatrix(10, layers=[128, 128], activation='tanh')
ff_network_amatrix_32_2 = QNetworkAmatrix(10, layers=[32, 32], activation='tanh')
ff_network_amatrix_32_3 = QNetworkAmatrix(10, layers=[32, 32, 32], activation='tanh')

In [ ]:
ff_network_amatrix_losses = qnetwork_pretrain(ff_network_amatrix, shuffle(data), epochs=10)

In [ ]:
ff_network_amatrix_64_3_losses = qnetwork_pretrain(ff_network_amatrix_64_3, shuffle(data), epochs=10)

In [ ]:
ff_network_amatrix_128_2_losses = qnetwork_pretrain(ff_network_amatrix_128_2, shuffle(data), epochs=10)

In [ ]:
ff_network_amatrix_32_2_losses = qnetwork_pretrain(ff_network_amatrix_32_2, shuffle(data), epochs=10)

In [ ]:
ff_network_amatrix_32_3_losses = qnetwork_pretrain(ff_network_amatrix_32_3, shuffle(data), epochs=10)

In [ ]:
losses_layers = np.array([ff_network_amatrix_losses,
                          ff_network_amatrix_32_2_losses,
                          ff_network_amatrix_32_3_losses,
                          ff_network_amatrix_64_3_losses,
                          ff_network_amatrix_128_2_losses]).transpose()

losses_layers_df = pd.DataFrame(data=losses_layers, columns=['64x2', '32x2', '32x3', '64x3', '128x2'])
losses_layers_df.to_csv('../logs/pre_train_data/layers_comparison.csv')

In [ ]:
losses_layers_df = pd.read_csv('../logs/pre_train_data/layers_comparison.csv')

ff_network_amatrix_losses = losses_layers_df['64x2']
ff_network_amatrix_32_2_losses = losses_layers_df['32x2']
ff_network_amatrix_32_3_losses = losses_layers_df['32x3']
ff_network_amatrix_64_3_losses = losses_layers_df['64x3']
ff_network_amatrix_128_2_losses = losses_layers_df['128x2']

x = range(2, 11)
fsize = 16
plt.figure(figsize=(13, 7))
plt.plot(x, ff_network_amatrix_losses[1:], label='64x2 layers')
plt.plot(x, ff_network_amatrix_32_2_losses[1:], label='32x2 layers')
plt.plot(x, ff_network_amatrix_32_3_losses[1:], label='32x3 layers')
plt.plot(x, ff_network_amatrix_128_2_losses[1:], label='128x2')
plt.plot(x, ff_network_amatrix_64_3_losses[1:], label='64x3')
plt.legend(prop={'size': fsize})
plt.xlabel('Epoch', fontsize=fsize)
plt.xticks(x)
plt.grid()
plt.ylabel('MSE', fontsize=fsize)
plt.title('Comparison of FF network configurations by pre-training speed', fontsize=fsize)
plt.show()

In [ ]:
ff_network_simple = QNetwork(10, activation='relu', layers=[64, 64])
ff_network_amatrix = QNetworkAmatrix(10, activation='relu', layers=[64, 64])

In [ ]:
data_full_network = data[data['pkg_id'] < 5000]

In [ ]:
ff_network_simple_losses = qnetwork_pretrain(ff_network_simple, shuffle(data_full_network), epochs=20)

In [ ]:
ff_network_amatrix_losses = qnetwork_pretrain(ff_network_amatrix, shuffle(data), epochs=20)

In [ ]:
ff_network_amatrix_adam_losses = qnetwork_pretrain(ff_network_amatrix, shuffle(data),
                                                   optimizer='adam', epochs=20, save_net=False)

In [ ]:
ff_network_amatrix_adagrad_losses = qnetwork_pretrain(ff_network_amatrix, shuffle(data),
                                                      optimizer='adagrad', epochs=20, save_net=False)

In [ ]:
ff_network_amatrix_adadelta_losses = qnetwork_pretrain(ff_network_amatrix, shuffle(data),
                                                       optimizer='adadelta', epochs=20, save_net=False)

In [ ]:
losses = np.array([ff_network_amatrix_losses, ff_network_amatrix_adam_losses,
                     ff_network_amatrix_adagrad_losses, ff_network_amatrix_adadelta_losses]).transpose()

In [ ]:
losses_df = pd.DataFrame(data=losses, columns=['rmsprop', 'adam', 'adagrad', 'adadelta'])
losses_df.to_csv('../logs/pre_train_data/optimizer_comparison.csv')

In [ ]:
losses_df = pd.read_csv('../logs/pre_train_data/optimizer_comparison.csv')

ff_network_amatrix_losses = losses_df['rmsprop'][:10]
ff_network_amatrix_adam_losses = losses_df['adam'][:10]
ff_network_amatrix_adagrad_losses = losses_df['adagrad'][:10]
ff_network_amatrix_adadelta_losses = losses_df['adadelta'][:10]

x = range(1, 11)
fsize = 25
plt.figure(figsize=(13, 14))
plt.plot(x, ff_network_amatrix_losses, label='RMSProp')
plt.plot(x, ff_network_amatrix_adam_losses, label='Adam')
plt.plot(x, ff_network_amatrix_adagrad_losses, label='AdaGrad')
plt.plot(x, ff_network_amatrix_adadelta_losses, label='AdaDelta')
plt.legend(prop={'size': fsize})
plt.xlabel('Номер эпохи', fontsize=fsize)
plt.xticks(x)
plt.grid()
plt.ylabel('MSE', fontsize=fsize)
plt.title('Сравнение качества предобучения с разными\n алгоритмами оптимизации (ReLU слои)', fontsize=fsize)
plt.show()

In [ ]:
ff_network_tanh_amatrix = QNetworkAmatrix(10, activation='tanh')

In [ ]:
ff_network_tanh_amatrix_losses = qnetwork_pretrain(ff_network_tanh_amatrix, shuffle(data), epochs=10, save_net=False)

In [ ]:
ff_network_tanh_amatrix_adam_losses = qnetwork_pretrain(ff_network_tanh_amatrix, shuffle(data), epochs=10,
                                                        optimizer='adam', save_net=False)

In [ ]:
ff_network_tanh_amatrix_adagrad_losses = qnetwork_pretrain(ff_network_tanh_amatrix, shuffle(data), epochs=10,
                                                           optimizer='adagrad', save_net=False)

In [ ]:
ff_network_tanh_amatrix_adadelta_losses = qnetwork_pretrain(ff_network_tanh_amatrix, shuffle(data), epochs=10,
                                                            optimizer='adadelta', save_net=False)

In [ ]:
losses_tanh = np.array([ff_network_tanh_amatrix_losses, ff_network_tanh_amatrix_adam_losses,
                        ff_network_tanh_amatrix_adagrad_losses, ff_network_tanh_amatrix_adadelta_losses]).transpose()
losses_tanh_df = pd.DataFrame(data=losses_tanh, columns=['rmsprop', 'adam', 'adagrad', 'adadelta'])
losses_tanh_df.to_csv('../logs/pre_train_data/optimizer_comparison_tanh.csv')

In [ ]:
color_c = cycler('color', ['k'])
style_c = cycler('linestyle', ['-', '--', ':', '-.'])
markr_c = cycler('marker', ['', '.', 'o'])
c_cms = color_c * markr_c * style_c

losses_tanh_df = pd.read_csv('../logs/pre_train_data/optimizer_comparison_tanh.csv')

ff_network_tanh_amatrix_losses = losses_tanh_df['rmsprop']
ff_network_tanh_amatrix_adam_losses = losses_tanh_df['adam']
ff_network_tanh_amatrix_adagrad_losses = losses_tanh_df['adagrad']
ff_network_tanh_amatrix_adadelta_losses = losses_tanh_df['adadelta']

x = range(1, 11)
fsize = 14
ticksize = 10
lw=3
f = plt.figure(figsize=(7, 5))
plt.plot(x, ff_network_tanh_amatrix_losses, label='RMSProp', linewidth=lw, alpha=0.6)#, color='k', linestyle='-')
plt.plot(x, ff_network_tanh_amatrix_adam_losses, label='Adam', linewidth=lw, alpha=0.6)#, color='k', linestyle='--')
plt.plot(x, ff_network_tanh_amatrix_adagrad_losses, label='AdaGrad', linewidth=lw)#, color='k', linestyle=':')
plt.plot(x, ff_network_tanh_amatrix_adadelta_losses, label='AdaDelta', linewidth=lw)#, color='k', linestyle='-.')
plt.legend(prop={'size': 14})
plt.xlabel('Epoch', fontsize=fsize)
plt.xticks(x)
plt.rc('xtick', labelsize=ticksize)
plt.rc('ytick', labelsize=ticksize)
plt.grid()
plt.ylabel('MSE', fontsize=fsize)
plt.ylim(0, 1)
plt.title('Comparison of optimization algorithms by pre-training speed', fontsize=fsize)
plt.show()

plt.savefig("../img/opt-algos-pre-training-comparison.pdf", bbox_inches='tight')

## Dropout & stuff

In [ ]:
ff_network_amatrix_dropout_mid = QNetworkAmatrix(10, activation='relu', layers=[64, 'dropout', 64])
ff_network_amatrix_dropout_end = QNetworkAmatrix(10, activation='relu', layers=[64, 64, 'dropout'])
ff_network_amatrix_dropout_both = QNetworkAmatrix(10, activation='relu', layers=[64, 'dropout', 64, 'dropout'])

In [ ]:
ff_network_amatrix_dropout_mid_losses = qnetwork_pretrain(ff_network_amatrix_dropout_mid, shuffle(data), epochs=10,
                                                          optimizer='adagrad')

In [ ]:
ff_network_amatrix_dropout_end_losses = qnetwork_pretrain(ff_network_amatrix_dropout_end, shuffle(data), epochs=10,
                                                          optimizer='adagrad')

In [ ]:
ff_network_amatrix_dropout_both_losses = qnetwork_pretrain(ff_network_amatrix_dropout_both, shuffle(data), epochs=10,
                                                           optimizer='adagrad')

## Embeddings 

In [ ]:
embedding = CachedEmbedding(LaplacianEigenmap, dim=4)

In [ ]:
embed_network_no_inp = QNetwork(10, activation='relu', layers=[64, 64], embedding_dim=embedding.dim)
embed_network_amatrix = QNetworkAmatrix(10, activation='relu', layers=[64, 64], embedding_dim=embedding.dim)

In [ ]:
embed_network_no_inp_losses = qnetwork_pretrain(embed_network_no_inp, shuffle(data), epochs=10,
                                                embedding=embedding)

In [ ]:
embed_network_amatrix_losses = qnetwork_pretrain(embed_network_amatrix, shuffle(data_full_network), epochs=10,
                                                 embedding=embedding)

In [ ]:
embed_network_no_inp_tanh = QNetwork(10, activation='tanh', layers=[64, 64], embedding_dim=embedding.dim)
embed_network_amatrix_tanh = QNetworkAmatrix(10, activation='tanh', layers=[64, 64], embedding_dim=embedding.dim)

In [ ]:
embed_network_no_inp_tanh_losses = qnetwork_pretrain(embed_network_no_inp_tanh, shuffle(data_full_network), epochs=20,
                                                     embedding=embedding)

In [ ]:
embed_network_amatrix_tanh_losses = qnetwork_pretrain(embed_network_amatrix_tanh, shuffle(data_full_network), epochs=20,
                                                      embedding=embedding)

In [ ]:
plot_losses({
    'no_inp_relu': embed_network_no_inp_losses,
    #'amatrix_relu': embed_network_amatrix_losses,
    #'no_inp_tanh': embed_network_no_inp_tanh_losses,
    #'amatrix_tanh': embed_network_amatrix_tanh_losses
})
embed_network_no_inp_losses

# Конвейеры

In [ ]:
conv_emb = CachedEmbedding(LaplacianEigenmap, dim=5)

In [ ]:
conveyor_network_ng_emb = QNetwork(22, scope='conveyor_test_ng', activation='relu', layers=[64, 64],
                                   embedding_dim=conv_emb.dim)
conveyor_network_ng_amatrix = QNetwork(22, scope='conveyor_test_ng', activation='tanh', layers=[64, 64],
                                       additional_inputs=[{'tag': 'amatrix'}])
conveyor_network_ng_full = QNetwork(22, scope='conveyor_test_ng', activation='tanh', layers=[64, 64],
                                    additional_inputs=[{'tag': 'amatrix'}, {'tag':'work_status'}])

In [ ]:
data_conv_ng = pd.read_csv('../logs/data_conveyor_gen_energy_test_oneinp.csv', index_col=0)

In [ ]:
data_conv_ng.head()

In [ ]:
conveyor_network_ng_emb_losses = qnetwork_pretrain(conveyor_network_ng_emb, shuffle(data_conv_ng), epochs=20,
                                                   embedding=conv_emb)

In [ ]:
conveyor_network_ng_amatrix_losses = qnetwork_pretrain(conveyor_network_ng_amatrix, shuffle(data_conv_ng), epochs=20)

In [ ]:
conveyor_network_ng_full_losses = qnetwork_pretrain(conveyor_network_ng_full, shuffle(data_conv_ng), epochs=20)

In [ ]:
plot_losses({
    'no_inp': conveyor_network_ng_emb_losses,
    #'amatrix': conveyor_network_ng_amatrix_losses,
    #'amatrix_work_status': conveyor_network_ng_full_losses,
})
conveyor_network_ng_emb_losses